In [2]:
import re
import os
from google.cloud import bigquery
#import urllib.parse
#import unicodedata
import csv
import itertools
from pattern.text.nl import singularize

In [6]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'C:\\Users\\KoeReu\\OneDrive - Blokker B.V\\Bureaublad\\Google BQ\\universal-analytics-nextail-baa80f5ed3cc.json'

client = bigquery.Client()

In [7]:
# create list of all letters, numbers and a space (" ")
all_letters = list('abcdefghijklmnopqrstuvqxyz1234567890 ')

# create list of tonal changes in Dutch
tone_change_lst = [('eau', 'o'), ('eau', 'oo'), ('ij', 'ei'), ('ei', 'ij'), ('ij', 'y'), ('ei', 'y'), 
                   ('y', 'ei'), ('y', 'ij'), ('au', 'ou'), ('ou', 'au'), ('g', 'ch'), ('ch', 'g'), 
                   ('ph', 'f'), ('sch', 'sg')]

In [8]:
def all_possible_misspellings(string: str):
    
    """Takes in a word and returns all possible misspellings in a set. This is done by the actions below. Input stofzuiger
    returns:
    - Removing letters - tofzuiger, sofzuiger, stfzuiger, stozuiger etc.
    - Repeating letters - sstofzuiger, sttofzuiger, stoofzuiger, stoffzuiger etc.
    - Creating typos - atofzuiger, btofzuiger, ctofzuiger, saofzuiger etc.
    - Switching adjacent letters - tsofzuiger, sotfzuiger, stfozuiger etc.
    - Rewriting tone changes - input airfryer returns airfrijer, airfreier etc.
    
    """
    
    
    string_lst = list(string)
    misspellings = []
                   
    for num, letter in enumerate(string_lst):
        
        # removal of letters
        misspellings.append(''.join(string_lst[:num] + string_lst[num + 1:]).strip())
        
        # repetition of letters
        misspellings.append(''.join(string_lst[:num] + [letter] * 2 + string_lst[num + 1:]))
        
        # create typos
        for repl_letter in all_letters:
            misspellings.append(''.join(string_lst[:num] + [repl_letter] + string_lst[num + 1:]).strip())
    
        # switching of adjacent letters
        if num == 0:
            misspellings.append(''.join([string_lst[num + 1]] + [string_lst[num]] + string_lst[num + 2:]))
        elif num < len(string_lst) - 1:
            misspellings.append(''.join(string_lst[:num] + [string_lst[num + 1]] + [string_lst[num]] + string_lst[num + 2:]))
            
    # change tone changes in Dutch (such as au: ou and ei: ij)
    for row in tone_change_lst:
        letter = row[0]
        repl_letter = row[1]
        misspellings.append(string.replace(letter, repl_letter))
            
    return set(misspellings)

In [9]:
def vkw(slug):
    
    """ Takes in category slug (e.g. stofzuigers or stofzuigers/philips) and returns the singular word and the 
    vkw (verkleinwoord) in a set (e.g. stofzuiger, stofzuigers, stofzuigertje, stofzuigertjes) """
    
    slug = slug.replace('-', ' ')
    #slug = slug.replace('/', ' ')
    slug_singular = singularize(slug)
    
    if re.match('.*pan$', slug_singular):
        vkw = slug_singular + 'netje'
        
    elif re.match('.*glas$', slug_singular):
        vkw = re.sub('glas', 'glaasje', slug_singular)
        
    elif re.match('.*kom$', slug_singular):
        vkw = re.sub('kom', 'kommetje', slug_singular)
                         
    elif re.match('.*m$', slug_singular):
        vkw = slug_singular + 'pje'
                     
    elif re.match('.*[k|t|p|s|d]$', slug_singular):
        vkw = slug_singular + 'je'
    
    elif re.match('.*[n|l|r]$', slug_singular):
        vkw = slug_singular + 'tje'
    
    try:
        return {slug, slug_singular, vkw, vkw + 's'}
    
    except:
        return {slug, slug_singular}

In [10]:
def extract_filters_from_url(clean_url: str):

    """This function returns all possible singular and plural words and verkleinwoorden for each category.
        
    URL /stofzuigers returns
    - stofzuiger
    - stofzuigers
    - stofzuigertje
    - stofzuigertjes
    
    URL /stofzuigers/philips returns:
    - philips stofzuiger
    - philips stofzuigers
    - philips stofzuigertje
    - philips stofzuigertjes
    - stofzuiger philips
    - stofzuigers philips
    - stofzuigertje philips
    - stofzuigertjes philips
    """
        
    # split clean URL from character two and beyond on slash (/) and underscore (_)
    splitted_clean_url = re.split('[\/_]', clean_url[1:])
    all_words = vkw(splitted_clean_url[0])
    
    if len(splitted_clean_url) == 1:
        return all_words
    
    elif len(splitted_clean_url) >= 2:
        r_set = set()
        r = itertools.chain(itertools.product(all_words, splitted_clean_url[1:]), itertools.product(splitted_clean_url[1:], all_words))
        for a, b in r:
            r_set.add(f'{a} {b}')
            
        return r_set
        
    # else: add third and beyond options

In [11]:
def create_misspellings(clean_url: str):
    """
    This function takes in a clean URL and returns all possible misspellings.
    
    URL /stofzuigers returns:
    - smofzuiger
    - stofzuigeas
    - st0fzuiger
    - atofzuigers
    - stof6uigers
    
    and 1,800 other possible misspellings
    """ 
    
    misspellings = []
    
    for wrd in extract_filters_from_url(clean_url):
        
        misspellings.extend(all_possible_misspellings(wrd))
        
    return set(misspellings)

In [12]:
# don't forget to change period here!

QUERY = (
    r"""
DECLARE re_str_brands STRING DEFAULT (SELECT CONCAT('/(', STRING_AGG(SUBSTRING(brand_slug, 2), '|'), ')$') FROM `universal-analytics-nextail.brands.brands_and_brand_slugs`);

SELECT LOWER(REGEXP_EXTRACT((SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'clean_url'), '(/[^/]+)')),
COUNT(*)
FROM `universal-analytics-nextail.analytics_283141299.events_*` 
WHERE --_TABLE_SUFFIX BETWEEN '20240601' AND FORMAT_DATE('%Y%m%d', CURRENT_DATE()) AND
event_name = 'page_view' AND 
(SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'page_type') = 'category' AND
NOT REGEXP_CONTAINS((SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'clean_url'), '^\\/(wk\\d|winter|bc|douwe|trend|folder)') AND
NOT REGEXP_CONTAINS((SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'clean_url'), re_str_brands) AND
(SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'clean_url') NOT LIKE '%-en-%'
GROUP BY 1
ORDER BY 2 DESC

    """
        )

query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

In [13]:
og_dict = {}

for clean_url, pv_count in query_job:
    if clean_url :
        og_dict[clean_url] = pv_count

In [24]:
result_dict = {}

for k, v in og_dict.items():
    if len(k) > 3:
        result_dict[k] = create_misspellings(k)

#### Read Nikki's redirects

In [26]:
# Read Nikki's redirects
%cd C:\Users\KoeReu\OneDrive - Blokker B.V\Documenten\Nikkis search redirects

sd = []

with open('search_driven_redirects.csv', mode = 'r', newline = '\n') as f:
    csv_reader = csv.reader(f, delimiter = ';')
    next(csv_reader) # skip first row
    # columns ['Categorie', 'Cat. ID', 'Live per', 'Zoekwoorden', '']
    
    for cat, cat_id, date, _, search_queries in csv_reader:
        
        sq = re.split(',\s?', search_queries)
        sd.append([cat, cat_id, [x.strip() for x in sq]])

C:\Users\KoeReu\OneDrive - Blokker B.V\Documenten\Nikkis search redirects


In [27]:
# unique redirects to all url's (e.g. prullenbakdn, prullenbak, prullenbakken, prullenbk, prullenbakk etc.)
search_redirects = set(list(itertools.chain.from_iterable([x[2] for x in sd])))
print(len(search_redirects))

9547


In [19]:
# A lot of erractic search behaviour was found in the dataset. This seemed to be generated by bot traffic, where
# obscure (not natural looking) search queries were triggered in high volumes. Another trait was that these search queries
# were highly volatile by nature. In the query below, all search queries per day are cross joined with all dates. This way,
# search queries with high volatility can be disinguished and possibly ignored.


QUERY = (
    r"""
DECLARE start_date DATE DEFAULT '2024-02-01';
DECLARE end_date DATE DEFAULT DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY);

WITH cte AS (SELECT event_date, 
TRIM((SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'search_term')) AS search_term,
COUNT(*) AS search_volume
FROM `universal-analytics-nextail.analytics_283141299.events_*`
WHERE _TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', start_date) AND 
FORMAT_DATE('%Y%m%d', end_date) AND
event_name = 'interaction_search' AND
(SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'search_type') = 'standard_search'
GROUP BY 1, 2),

cte_2 AS (SELECT * -- dates, cte.search_term, cte.search_volume
FROM UNNEST(GENERATE_DATE_ARRAY(start_date, end_date, INTERVAL 1 DAY)) AS dates
CROSS JOIN (SELECT DISTINCT(search_term) FROM cte)),

cte_3 AS (SELECT cte_2.dates, cte_2.search_term, COALESCE(cte.search_volume, 0) AS sv FROM cte_2
LEFT JOIN cte ON cte_2.dates = PARSE_DATE('%Y%m%d', cte.event_date) AND cte_2.search_term = cte.search_term)

SELECT cte_3.search_term,
SUM(sv) AS sv_sum,
AVG(sv) AS sv_avg,
STDDEV(sv) AS sv_std,
DATE_DIFF(end_date, start_date, DAY) AS delta_days, 
SQRT(DATE_DIFF(end_date, start_date, DAY)) AS sqrt_dd,
STDDEV(sv) * SQRT(DATE_DIFF(end_date, start_date, DAY)) AS volatility,
CASE WHEN STDDEV(sv) * SQRT(DATE_DIFF(end_date, start_date, DAY)) > 500 THEN true ELSE false END AS highly_volatile
FROM cte_3
GROUP BY 1
ORDER BY 2 DESC
    """
        )

query_job = client.query(QUERY)  # API request
rows_sq = query_job.result()  # Waits for query to finish

In [20]:
all_url_redirects = list(itertools.chain.from_iterable(result_dict.values())) # create list of nested lists
ga4_sq = {}
all_kw = {}
sq_lookup_set = set()

for search_query, sv_sum, sv_avg, sv_std, delta_days, sqrt_dd, volatility, highly_volatile in query_job:
    
    if search_query and search_query not in ga4_sq:
        ga4_sq[search_query] = 0
    
    # the dictionary here is filled with search volumes for keywords without high volatility
    if search_query and highly_volatile == False:
        ga4_sq[search_query] += sv_sum
               
    if search_query:
        all_kw[search_query] = [sv_sum, sv_avg, sv_std, delta_days, sqrt_dd, volatility, highly_volatile]

In [28]:
# create list for exporting to CSV
to_csv = []
min_search_volume = 1 # filter for minimum search volume per query

for k, v in result_dict.items():
    result_sd = [] # search redirects
    result_nsd = [] # search non-directs
    
    for sq in v:
        # search volume excluding high volatility keywords
        search_volume = ga4_sq.get(sq, False)
        
        if search_volume and search_volume >= min_search_volume:
            if sq in search_redirects:
                result_sd.append((sq, search_volume))
                
            elif sq not in search_redirects:
                result_nsd.append((sq, search_volume))
                           
    result_sd = sorted(result_sd, key = lambda x: x[1], reverse = True)
    result_nsd = sorted(result_nsd, key = lambda x: x[1], reverse = True)
    
    if result_nsd:
        to_csv.append([k, result_nsd, result_sd, sum([x[1] for x in result_nsd])])

In [29]:
%pwd

'C:\\Users\\KoeReu\\OneDrive - Blokker B.V\\Documenten\\Nikkis search redirects'

In [30]:
with open('potentieel_ontbrekende_redirects_with_self_created_misspellings.csv', 'w', newline = '\n', encoding = 'utf-8') as f:
    csv_writer = csv.writer(f, delimiter = ';')
    csv_writer.writerow(['url', 'search_queries_non_redirected', 'search_queries_non_redirected_w_volume', 'search_queries_redirected', 'total_search_volume'])
    
    for k, result_nsd, result_sd, search_volume in sorted(to_csv, key = lambda x: x[-1], reverse = True):
        csv_writer.writerow([k, ', '.join([f'[{x[0]}]' for x in result_nsd]), result_nsd, result_sd, search_volume])

In [31]:
cols = 'search_query, sv_sum, sv_avg, sv_std, delta_days, sqrt_dd, volatility, highly_volatile'

with open('all_search_queries.csv', 'w', encoding = 'utf-8', newline = '\n') as f:
    csv_writer = csv.writer(f, delimiter = ';')
    csv_writer.writerow([x.strip() for x in cols.split(',')])
    
    for num, (k, v) in enumerate(all_kw.items()):
        
        # rewrites floats with dot notation to comma notation (1.99 to 1,99)
        w = ["{:.2f}".format(x).replace('.', ',') if type(x) == float else x for x in v]
        
        if num > 10000:
            break
        else:
            csv_writer.writerow([k] + w)